## Test task. Wiki Parser for 50 pages.

##### Задача:
* Написать парсер 50 заданных страниц Википедии, из которых надо выпарсить прямую ссылку на видеофайл и метаданные — формат видеофайла и его длительность. Дополнительно перевести размер в мегабайты и длительность в секунды.


In [1]:
# импортируем библиотеки
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
# создадим список с полными адресами страниц, которые будем парсить
import docx2txt
text = docx2txt.process('C:\\Users\\lenovo\\Documents\\yndx_test.docx') #локальный путь
text = text.split('\n\n')
new_list=[]
for i in text:
    k = 'https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%B9%D0%BB:'+i
    new_list.append(k)
new_list[0:5]

['https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%B9%D0%BB:Shutter_speed_04.ogv',
 'https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%B9%D0%BB:Keilriemenvariator_Matador.webm',
 'https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%B9%D0%BB:Brain_Both_Healthy_and_Diseased.webm',
 'https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%B9%D0%BB:Motorcycle_show_at_Motodays_2017.webm',
 'https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%B9%D0%BB:Targeting-of-FzrCdh1-for-timely-activation-of-the-APCC-at-the-centrosome-during-mitotic-exit-ncomms12607-s4.ogv']

In [3]:
host = 'https://ru.wikipedia.org/'
# url = 'https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%B9%D0%BB:Using-Existing-Drugs-as-Leads-for-Broad-Spectrum-Anthelmintics-Targeting-Protein-Kinases-ppat.1003149.s018.ogv'
headers = {'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
          'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}

In [4]:
# получим текстовое представление
def get_html(url):
    r = requests.get(url, headers=headers).text
    return r

# парсер для прямой ссылки
def parse2(some_url):
    html=get_html(some_url)
    soup = BeautifulSoup(html, 'lxml') # собственно парсер всей страницы
    meta = soup.find('div', attrs={'class':'fullMedia'}) # ищем блок со ссылкой
    link = meta.find('a').get('href') # выделяем ссылку
    return link

In [5]:
# создадим список прямых ссылок на видео для всех сайтов
links_list=[]
for site in new_list:
    a = parse2(site)
    links_list.append(a)
#links_list

In [6]:

#bad_list=[] # это список для нестандартных страниц. Оттуда потом нужно будет забрать данные иначе, чем для основных
def bad_parse(some_url):
    html=get_html(some_url)
    soup = BeautifulSoup(html, 'lxml') # собственно парсер всей страницы
    meta = soup.find('table', class_='wikitable filehistory')
    tabs = meta.find_all('tr')
    td_tab =[]
    for tr in tabs:
        cols = tr.findAll('td')

        for td in cols:
            f_td = td.contents[0]
            td_tab.append(f_td)
            for span in td.findAll('span'):
                f_span = span.text
                td_tab.append(f_span)

                
    return td_tab


In [7]:
bad_list=[] 
for i in new_list:
    try:
        bad_parse(i)
    except Exception:
        bad_list.append(i)
        
        
bad_list  # это список для нестандартных страниц. Оттуда потом нужно будет забрать данные иначе, чем для основных

['https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%B9%D0%BB:Using-Existing-Drugs-as-Leads-for-Broad-Spectrum-Anthelmintics-Targeting-Protein-Kinases-ppat.1003149.s018.ogv',
 'https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%B9%D0%BB:Nad_liticon,_Dubravica-Jurjevići_131703.webm',
 'https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%B9%D0%BB:Peliški_kanal_sa_vidikovca5106.ogv']

In [8]:
def parse(some_url):
    #print(some_url)
    html=get_html(some_url)
    soup = BeautifulSoup(html, 'lxml') # собственно парсер всей страницы
    meta = soup.find('table', class_='wikitable filehistory')
    tabs = meta.find_all('tr')
    td_tab =[]
    for tr in tabs:
        cols = tr.findAll('td')
        try:
            for td in cols:
                f_td = td.contents[0]
                td_tab.append(f_td)
                for span in td.findAll('span'):
                    f_span = span.text
                    td_tab.append(f_span)
        except Exception:
            print(some_url)
            bad_list.append(some_url) # исключения добавляем в отдельный список для последующей обработки
            for td in cols:
                f_td = 'n/a'
                #bad_list.append(some_url) 
                td_tab.append(f_td)
                for span in td.findAll('span'):
                    f_span = span.text
                    td_tab.append(f_span)
                
    return td_tab


#line = parse('https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%B9%D0%BB:Astroglial-axonal-interactions-during-early-stages-of-myelination-in-mixed-cultures-using-in-vitro-1471-2202-15-59-S1.ogv')
#line = parse(url)
#bad_list

In [9]:
sizes = []
lengths = []
bad_size=[]  
bad_length=[]
exc1 = 'https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%B9%D0%BB:Using-Existing-Drugs-as-Leads-for-Broad-Spectrum-Anthelmintics-Targeting-Protein-Kinases-ppat.1003149.s018.ogv'
for i in new_list:
    if i not in bad_list:
        k = parse(i)
        #print(k)
        i_size = k[-3] # сначала из нормальных страниц забираем размер
        i_size = i_size.replace("(", "").replace(')','')
        sizes.append(i_size)
        # и с нормальных страниц забираем длину
        i_length = ((k[-4]).split(', '))[0]
        lengths.append(i_length)
        #print(i_length)
    elif i == exc1:
        k = parse(i)
        i_size = k[-10] # сначала из нормальных страниц забираем размер
        i_size = i_size.replace("(", "").replace(')','')
        sizes.append(i_size)
        i_length = ((k[-11]).split(', '))[0]
        lengths.append(i_length)
    # теперь для нестандартных страниц:    
    else: 
        k = parse(i)
        i_size = k[-11]
        #print(k)
        i_size = i_size.replace("(", "").replace(')','')
        i_size=(i_size.split(', '))[0]
        sizes.append(i_size)
        bad_size.append(i_size) # это для проверки и отладки
        
        # забираем длину
        i_length = ((k[-12]).split(', '))[0]
        #print(i_length)
        bad_length.append(i_length)
        lengths.append(i_length)

#sizes
#lengths
#bad_length

https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%B9%D0%BB:Using-Existing-Drugs-as-Leads-for-Broad-Spectrum-Anthelmintics-Targeting-Protein-Kinases-ppat.1003149.s018.ogv
https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%B9%D0%BB:Nad_liticon,_Dubravica-Jurjevići_131703.webm
https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%B9%D0%BB:Peliški_kanal_sa_vidikovca5106.ogv


In [10]:
df = pd.DataFrame(new_list, links_list) #фрейм с сайтом и ссылкой на видео
df['time'] = lengths
df['size'] = sizes
df.reset_index(inplace=True)
df.columns=['video_link', 'page', 'raw_time', 'raw_size']
df.head()

,video_link,page,raw_time,raw_size
0,//upload.wikimedia.org/wikipedia/commons/5/5d/...,https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%...,25 с,"54,75 Мб"
1,//upload.wikimedia.org/wikipedia/commons/d/d0/...,https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%...,23 с,"2,82 Мб"
2,//upload.wikimedia.org/wikipedia/commons/d/da/...,https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%...,15 с,"3,32 Мб"
3,//upload.wikimedia.org/wikipedia/commons/1/18/...,https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%...,35 с,"8,69 Мб"
4,//upload.wikimedia.org/wikipedia/commons/7/70/...,https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%...,12 с,217 Кб


Переведем минуты в секунды и Кб и Мб

In [11]:
df2 = df.copy()

import re
# разобьем колонку с временем на 3 составляющие
df2['raw_time_new'] = df2['raw_time'].apply(lambda x: x.replace(' с', ''))
df2['splitted_time'] = df2['raw_time_new'].apply(lambda x: re.split('ч |м ', x))

# создаем отдельные столбцы с секундами и часами и минутами (если есть)
df2['seconds'] = df2['splitted_time'].apply(lambda x: x[-1])
df2['minutes'] = df2['splitted_time'].apply(lambda x: x[-2] if len(x) > 1 else 0)
df2['hours'] = df2['splitted_time'].apply(lambda x: x[-3] if len(x) > 2 else 0)

df2.head()

,video_link,page,raw_time,raw_size,raw_time_new,splitted_time,seconds,minutes,hours
0,//upload.wikimedia.org/wikipedia/commons/5/5d/...,https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%...,25 с,"54,75 Мб",25,[25],25,0,0
1,//upload.wikimedia.org/wikipedia/commons/d/d0/...,https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%...,23 с,"2,82 Мб",23,[23],23,0,0
2,//upload.wikimedia.org/wikipedia/commons/d/da/...,https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%...,15 с,"3,32 Мб",15,[15],15,0,0
3,//upload.wikimedia.org/wikipedia/commons/1/18/...,https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%...,35 с,"8,69 Мб",35,[35],35,0,0
4,//upload.wikimedia.org/wikipedia/commons/7/70/...,https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%...,12 с,217 Кб,12,[12],12,0,0


In [12]:
def comma_replace(df, cols_list):
    for col in cols_list:
        df[col] = df[col].apply(lambda x: x.replace(',', '.'))

comma_replace(df2, ['seconds'])

In [13]:

def make_num(df, cols_list): # ф-ция для перевода нескольких столбцов фрейма в числовой формат
    for col in cols_list:
        #df[col] = df[col].apply(lambda x: x.replace(',', '.'))
        df[col] = pd.to_numeric(df[col])
        
make_num(df2, ['seconds', 'minutes', 'hours']) 

In [14]:
df2['total_seconds'] = df2['hours']*60*60 + df2['minutes']*60 +df2['seconds']
df2.head()

,video_link,page,raw_time,raw_size,raw_time_new,splitted_time,seconds,minutes,hours,total_seconds
0,//upload.wikimedia.org/wikipedia/commons/5/5d/...,https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%...,25 с,"54,75 Мб",25,[25],25.0,0,0,25.0
1,//upload.wikimedia.org/wikipedia/commons/d/d0/...,https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%...,23 с,"2,82 Мб",23,[23],23.0,0,0,23.0
2,//upload.wikimedia.org/wikipedia/commons/d/da/...,https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%...,15 с,"3,32 Мб",15,[15],15.0,0,0,15.0
3,//upload.wikimedia.org/wikipedia/commons/1/18/...,https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%...,35 с,"8,69 Мб",35,[35],35.0,0,0,35.0
4,//upload.wikimedia.org/wikipedia/commons/7/70/...,https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%...,12 с,217 Кб,12,[12],12.0,0,0,12.0


In [15]:
# для перевода Кб в Мб поделиб Кб на 1000

df2['format'] = df2['raw_size'].apply(lambda x: (x.split(' '))[-1])
df2['value']  = df2['raw_size'].apply(lambda x: (x.split(' '))[-2])
df2.head()

,video_link,page,raw_time,raw_size,raw_time_new,splitted_time,seconds,minutes,hours,total_seconds,format,value
0,//upload.wikimedia.org/wikipedia/commons/5/5d/...,https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%...,25 с,"54,75 Мб",25,[25],25.0,0,0,25.0,Мб,"54,75"
1,//upload.wikimedia.org/wikipedia/commons/d/d0/...,https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%...,23 с,"2,82 Мб",23,[23],23.0,0,0,23.0,Мб,"2,82"
2,//upload.wikimedia.org/wikipedia/commons/d/da/...,https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%...,15 с,"3,32 Мб",15,[15],15.0,0,0,15.0,Мб,"3,32"
3,//upload.wikimedia.org/wikipedia/commons/1/18/...,https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%...,35 с,"8,69 Мб",35,[35],35.0,0,0,35.0,Мб,"8,69"
4,//upload.wikimedia.org/wikipedia/commons/7/70/...,https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%...,12 с,217 Кб,12,[12],12.0,0,0,12.0,Кб,217


In [16]:
# df2['value'] = df2['value'].apply(lambda x: x.replace(',', '.'))
comma_replace(df2, ['value'])
make_num(df2, ['value'])

In [17]:
def new_val(row):
    if row['format'] == 'Мб':
        res = row['value']
    else:
        res = row['value']/1000
    return res          
         
df2['new_value'] = df2.apply(new_val, axis=1)

In [18]:
final_df = df2.drop(['splitted_time', 'seconds', 'minutes', 'hours', 'format', 'value', 'raw_time_new'], axis=1)
final_df.columns = ['video_link', 'page', 'raw_time', 'raw_size', 'total_seconds', 'total_Mb_size']
final_df.head()

,video_link,page,raw_time,raw_size,total_seconds,total_Mb_size
0,//upload.wikimedia.org/wikipedia/commons/5/5d/...,https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%...,25 с,"54,75 Мб",25.0,54.750
1,//upload.wikimedia.org/wikipedia/commons/d/d0/...,https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%...,23 с,"2,82 Мб",23.0,2.820
2,//upload.wikimedia.org/wikipedia/commons/d/da/...,https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%...,15 с,"3,32 Мб",15.0,3.320
3,//upload.wikimedia.org/wikipedia/commons/1/18/...,https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%...,35 с,"8,69 Мб",35.0,8.690
4,//upload.wikimedia.org/wikipedia/commons/7/70/...,https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%...,12 с,217 Кб,12.0,0.217


In [19]:
final_df.to_excel('wikiparser.xlsx') # выгружаем в эксель

##### В результирующей таблице:
* video_link - прямая ссылка на видео
* page - ссылка на страницу
* raw_time - длительность видео в исходном формате (текстовом)
* raw_size - размер видео в исходном формате (текстовом)
* total_seconds - длительность видео в секундах
* total_Mb_size - размер видео в мегабайтах